In [ ]:
pip install simpy

In [8]:
import simpy
import random
import time

class SingleBulbGame:
    def __init__(self, env):  # Fixed __init__ method
        self.env = env
        self.correct_switch = random.choice(['A', 'B', 'C'])  # Randomly select the correct switch
        self.game_over = False
        self.start_time = time.time()
        self.switch_durations = {'A': 0, 'B': 0, 'C': 0}  # To store how long each switch was ON
        self.current_switch = None  # To track the currently active switch
        self.last_switch_time = self.start_time  # To track the last switch change time

    def check_time(self):
        """Check if 60 seconds have passed."""
        if time.time() - self.start_time >= 60:
            print("\n⏳ Time is up! You lost!\n")
            self.game_over = True
            return True
        return False

    def update_switch_duration(self, new_switch):
        """Update the duration of the current switch before switching to a new one."""
        if self.current_switch is not None:
            # Calculate the duration the current switch was ON
            duration = time.time() - self.last_switch_time
            self.switch_durations[self.current_switch] += duration
        self.current_switch = new_switch
        self.last_switch_time = time.time()

    def player_actions(self, env):
        """Player chooses switches and observes the bulb."""
        print("-----------------------Rules-----------------------")
        print("You have 3 switches (A, B, C) and 1 bulb.")
        print("You can toggle switches, but you can check the bulb only ONCE.")
        print("You have 60 seconds to complete the game!\n")

        if self.check_time():
            return

        # Step 1: Turn ON one switch for a few seconds, then turn it OFF
        switch1 = input("Turn ON one switch for a few seconds, then turn it OFF (Choose A, B, or C): ").strip().upper()
        self.update_switch_duration(switch1)  # Switch is ON
        yield env.timeout(5)  # Simulate 5 seconds of delay
        self.update_switch_duration(None)  # Switch is OFF

        if self.check_time():
            return

        # Step 2: Turn ON another switch and leave it ON
        switch2 = input("Now turn ON another switch and leave it ON (Choose a different switch): ").strip().upper()
        self.update_switch_duration(switch2)  # Switch is ON

        print("\nWaiting 55 seconds before you can check the bulb...\n")
        yield env.timeout(55)  # Simulate 55 seconds of delay
        self.update_switch_duration(None)  # Switch is OFF (bulb is cold)

        if self.check_time():
            return

        print(f"1. bulb is ON, so it belongs to switch {switch2}.")
        print(f"2. bulb is warm, so it belongs to switch {switch1}.")
        print(f"3. bulb is OFF and cold, so it belongs to the remaining switch.\n")

        # Step 4: Player guesses the correct switch
        print("\nHURRY! You have only *5 SECONDS* to guess the correct switch!")
        user_guess = input("Enter the switch (A, B, or C): ").strip().upper()

        if self.check_time():
            return

        # Step 5: Check if the guess is correct
        if user_guess == self.correct_switch:
            print("\nCongratulations! You correctly identified the switch!")
        else:
            print("\nIncorrect! The correct switch was:", self.correct_switch)

        self.game_over = True

# Run the Simulation
env = simpy.Environment()
game = SingleBulbGame(env)  # Now correctly initializes
env.process(game.player_actions(env))
env.run()

-----------------------Rules-----------------------
You have 3 switches (A, B, C) and 1 bulb.
You can toggle switches, but you can check the bulb only ONCE.
You have 60 seconds to complete the game!

Turn ON one switch for a few seconds, then turn it OFF (Choose A, B, or C): A
Now turn ON another switch and leave it ON (Choose a different switch): B

Waiting 55 seconds before you can check the bulb...

1. bulb is ON, so it belongs to switch B.
2. bulb is warm, so it belongs to switch A.
3. bulb is OFF and cold, so it belongs to the remaining switch.


HURRY! You have only *5 SECONDS* to guess the correct switch!
Enter the switch (A, B, or C): C

Congratulations! You correctly identified the switch!
